# robots.txt

# Scraper

In [ ]:
import scrapy
import scrapy.crawler as crawler
from bs4 import BeautifulSoup

from scrapy.crawler import CrawlerProcess

class GumtreeApartmentsSpider(scrapy.Spider):
    name = 'gumtreeapartmentsspider'
    start_urls = [
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-'+str(i)+'/v1c9073l3200001p'+str(i)  for i in range(2,4)
        ]
    start_urls.append(
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1'
    )
    found_apartments = []
   
    custom_settings = {
        'DOWNLOAD_DELAY': '4.0', #obejście Fail2Ban
        'ROBOTSTXT_OBEY' : True
    }

    top_url = 'https://www.gumtree.pl'
    def parse(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        link_tabs = soup.findAll("div", {"class": "result-link"})
        item_urls = []
        next_urls = []
        for tab in link_tabs:
            hrefs = tab.findAll("a", {"class": "href-link"})
            for h in hrefs:
                item_urls.append(self.top_url + h["href"]) #dopisuje 'https://www.gumtree.pl', bo otrzymany adres jest względny, przygotowuje listę wszystkich linków
            
        for item_url in item_urls:
            yield scrapy.Request(item_url, self.parse_item)
        
    def parse_item(self, response): #item_url - odwiedzanie strony, #self.parse_item - przetworzenie przy pomocy funkcji
        #Courtesy of Mr Sebastian Muraszewski
        found_apartments = []
        soup = BeautifulSoup(response.text, 'html.parser') 
        apartments = soup.find('div', {'class': 'vip-header-and-details'})
        apartment_details = dict()
        apartment_details['Nazwa ogłoszenia'] = apartments.find('span', class_ = 'myAdTitle').text
        apartment_details['Cena'] = apartments.find('span', class_ = 'amount').text.replace("\xa0"," ") #replace w celu usunięcia twardej spacji
        container = soup.find('ul', class_ = 'selMenu') #zebranie informacji z ramki do kontenera
        
        nazwy = container.findAll('span', class_ = 'name')
        szczegoly = container.findAll('span', class_ = 'value')
        apartment_details.update({name.text: value.text.strip() for name, value in zip(nazwy, szczegoly)}) #dodawanie elementów z ramki do listy i usunięcie whitespace'ów
        
        with open('plik.csv', 'a') as f:
            v = apartment_details.values()
            f.write('\t'.join(v))


In [ ]:
process = CrawlerProcess({
    'USER_AGENT': 'Testing MyCrawler (michal.korzycki@gmail.com)'
})
process.crawl(GumtreeApartmentsSpider)
process.start()